In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
import numpy as np
from activ.analytics import heatmap
from activ.clustering import UmapClusteringResults
from os.path import dirname, join
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize

In [2]:
f = h5py.File('/Users/ahyeon/Desktop/activ misc/predict_scores2.h5', 'r')

In [3]:
predictions = f['score'][:]
norm_predictions = f['norm_score'][:]
labels = f['clusters'][:]
umap_dims = f['umap_dimensions'][:]
cluster_sizes = f['cluster_sizes'][:]

In [ ]:
def accuracy(predicted, actual):
    correct = predicted-actual == 0
    tn_tp = len(np.where(correct))
    tn_tp_fn_fp = len(predicted)
    return tn_tp/tn_tp_fn_fp

In [ ]:
# specificity
iter_size = 50
allsens = np.zeros((len(cluster_sizes), iter_size))
allspec = np.zeros((len(cluster_sizes), iter_size))
allfscore = np.zeros((len(cluster_sizes), iter_size))
allacc = np.zeros((len(cluster_sizes), iter_size))
for ii,c in enumerate(cluster_sizes):
    all50mats = conf_alliter(c, norm_predictions, labels)
    sensitivities = []
    for iteration in range(iter_size):
        tps, tns, fps, fns = statistics(all50mats[iteration])
        
        tp_class = tps[ii]
        tn_class = tns[ii]
        fp_class = fps[ii]
        fn_class = fns[ii]
        
        sens = true_pos_rate(tp_class, fn_class)
        sensitivities.append(sens)
    allsens[ii] = sensitivities